In [23]:
import mediapipe as mp
import cv2
import numpy as np
import time
import os
import tensorflow as tf
from Fonction import *
import threading

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

mp_hands = mp.solutions.hands

In [39]:
model = tf.keras.models.load_model('test.h5')
# model.load_weights('train_results/lastweights.h5')

In [7]:
model.output_shape[1]

15

In [44]:
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume


def process():
    global data, ancien_lm, derive, proba_liss, flag, time_cmd
    # extraction des lms ########################################################
    hand_lm = lm_extraction(hand_resultats)
    lm_flat = hand_lm.copy()
    lm_flat = lm_flat.flatten()

    if hand_resultats.multi_hand_landmarks:
        # Dessin des landmarks ########################################################
        lm_drawing(image, hand_resultats)

        # calcule d'angles ########################################################
        angle = angle_process(hand_lm)
        cv2.putText(pop, 'angle :'+str(round(angle[3]*180/np.pi)), org=(0, 600), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.8, color=(150, 255, 0), thickness=1)

        # calcule d'angles ########################################################
        dist = distance_process(hand_lm)
        cv2.putText(pop, 'distance pouce/index :'+str(round(dist[6], 3)), org=(0, 650), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.8, color=(150, 255, 0), thickness=1)
    else:
        angle = np.zeros(15)
        dist = np.zeros(21)
    # mise en forme de la donnée (1,99) ########################################################
    data_frame = np.concatenate((lm_flat, angle.flatten(), dist.flatten()))
    # Sav des lm sur un vecteur temps de 20 echantion glissant
    data = np.concatenate(
        [data[1:], np.expand_dims(data_frame, axis=0)], axis=0)

    derive_actu = hand_lm-ancien_lm
    ancien_lm = hand_lm
    derive = np.concatenate(
        [derive[1:], np.expand_dims(derive_actu, axis=0)], axis=0)

    if hand_resultats.multi_hand_landmarks:
        id_hand = hand_resultats.multi_handedness[0].classification[0].label
        # mise en forme pour prédiction (1,99) ########################################################
        lmr = np.reshape(data, (1, 20, 99))

        # classification et calcul de temps ########################################################
        spred = time.time()
        vector_pred = model.predict_on_batch(lmr)[0]
        timepred = round((time.time()-spred)*1000, 3)

        # enregistrement de l'historique de prediction
        proba_liss = np.concatenate(
            [proba_liss[1:], np.expand_dims(vector_pred, axis=0)], axis=0)
        prob_filtre = np.mean(proba_liss, axis=0)

        # affichage du temps de prediction ########################################################
        cv2.putText(pop, 'temps classification :' + str(timepred)+" ms", org=(201, 120), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.5, color=(0, 255, 0), thickness=1)

        # AFFICHAGE ET TRAITEMENT DE LA PREDICTION  ########################################################
        # pred=np.argmax(vector_pred)
        # print(vector_pred)
        # print(pred," --> confidence ---> ",vector_pred[pred])
        proba_viz(prob_filtre, pop)

        # CALCUL DE VITESSE DE L'Index
        index_y_derive = np.mean(derive[50:-1, 8, 1])
        index_y_derive = index_y_derive*-1
        if vector_pred[10] > 0.85:
            volume = get_master_volume()
            mouv_v = round(index_y_derive*50, 2)
            if (time.time()-time_cmd) > 0.6:
                # Seuillage de l'activation
                if mouv_v > -0.5 and mouv_v < 0.5:
                    mouv_v = 0

                # Limitation de la modif de volume
                if mouv_v > 70:
                    mouv_v = 70
                if mouv_v < -50:
                    mouv_v = -50
                mouv_v = mouv_v/10
                new_vol = mouv_v+volume

                # limiation pour eviter les erreur systeme
                if new_vol > 1:
                    new_vol = 1
                elif new_vol < 0:
                    new_vol = 0

                set_master_volume(new_vol)
                print('volume : ', new_vol)
                time_cmd = time.time()
            cv2.putText(pop, 'vitesse du doigt :' + str(mouv_v), org=(201, 150), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.5, color=(0, 255, 0), thickness=1)
    else:
        id_hand = 'No hand'

    cv2.putText(pop, id_hand, org=(201, 160), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.5, color=(0, 255, 0), thickness=1)
    cv2.imshow('Resultat de détection', pop)
    cv2.imshow('hand Model', image)

In [45]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920/3)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080/3)
cap.set(cv2.CAP_PROP_FPS, 25)

# Parametre holitic
time_act = time.time()
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)

# Préparation des variable utils  ########################################################
fps = 0
fps_results = 0
cTime = time.time()
count = 0
data = np.zeros((20, 99))
derive = np.zeros((60, 21, 3))
ancien_lm = np.zeros((21, 3))
proba_liss = np.zeros((15, model.output_shape[1]))
time_cmd = time.time()
flag = 0


while cap.isOpened():
    ret, frame = cap.read()
    pop = np.zeros((800, 450, 3), dtype=np.uint8)

    # switch de canal couleur
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # flip pour un effect mirroir
    image = cv2.flip(image, 1)
    # detection
    image.flags.writeable = False
    hand_resultats = hands.process(image)
    image.flags.writeable = True
    # Retour en format BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    fps += 1
    if time.time()-cTime >= 1:
        fps_results = fps
        fps = 0
        cTime = time.time()
    # pTime=cTime
    cv2.putText(image, 'fps :'+str(fps_results), org=(50, 200), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1, color=(0, 255, 0), thickness=2)

    th1 = threading.Thread(target=process())
    th1.start()
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

c:\Users\manon\miniconda3\envs\env_ziane\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


volume :  0.7210000157356262
volume :  0.7210000157356262
volume :  0.7210000157356262


In [32]:
cap.release()
cv2.destroyAllWindows()

In [16]:
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import CLSCTX_ALL
def get_master_volume():
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume_object = interface.QueryInterface(IAudioEndpointVolume)
    return volume_object.GetMasterVolumeLevelScalar()

# Exemple d'utilisation
current_volume = get_master_volume()
print("Volume actuel:", current_volume)

Volume actuel: 0.1599999964237213


In [17]:
#si la valeur de ça est superieur a un seuil
classe_list[np.argmax(vector_pred)]


NameError: name 'classe_list' is not defined

In [19]:
import csv
with open('./classe.csv') as mon_fichier:
    mon_fichier_reader = csv.reader(mon_fichier, delimiter=',', quotechar='"')
    classe_list = [x for x in mon_fichier_reader]
classe_list=classe_list[0]

In [20]:
hand_resultats.multi_handedness[0].classification[0].label


NameError: name 'hand_resultats' is not defined

In [16]:
np.mean(np.array([[1,1,1],[2,2,2],[3,3,5]]),axis=0)

array([2.        , 2.        , 2.66666667])